In [2]:
import numpy as np
from numpy.linalg import inv
from sklearn.datasets.samples_generator import make_blobs, make_classification

# Data

In [53]:
N = 20
p = 3
X, y = make_classification(n_samples=N, n_features=p, n_informative=p, n_redundant=0, n_repeated=0, n_clusters_per_class=1)

## Logistic Regression

Logistic regression aims to model a binary outcome using a linear function in feature space. In particular, we model the log-odds as a linear combination of the predictor variables: $$\log{\frac{\mathrm{Pr}(y_i = 1|X)}{1-\mathrm{Pr}(y_i = 1|X)}} = \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + \ldots + \beta_p x_{ip} = \boldsymbol{\beta}^T\mathbf{x}_i$$, where $\mathbf{x}_i = (1, x_{i1}, \ldots, x_{ip})^T$.
One can show that this can be rewritten as: $$\mathrm{Pr}(y_i = 1|X)=\frac{\exp{\boldsymbol{\beta}^T\mathbf{x}_i}}{1+\exp{\boldsymbol{\beta}^T\mathbf{x}_i}}=\frac{1}{1+\exp{(-\boldsymbol{\beta}^T\mathbf{x}_i)}}$$.

Assuming that $y \sim Bin(\mathrm{Pr}(y_i = 1|X))$ we can use the binomial likelihood or binomial cross-entropy loss to fit the model: $$L(\boldsymbol{\beta};\mathbf{X}, \mathbf{y}) = \prod_i^n p^{y_i} (1-p)^{1-y_i}$$ where $\mathbf{X} = (\mathbf{x}_1, \ldots, \mathbf{x}_n)^T$ is a design matrix with the feature vectors as rows, $\mathbf{y}=(y_1,\ldots,y_n)^T$, and $p=\mathrm{Pr}(y_i = 1|X)$. We can obtain the log-likelihood as $$\ell(\boldsymbol{\beta};\mathbf{X}, \mathbf{y})=\sum_i^n y_i \log{(p)} + (1-y_i)\log{(1-p)}$$.

We now take the derivative with respect to $\boldsymbol{\beta}$ and set to zero. Note that $\frac{\partial p}{\partial \boldsymbol{\beta}} = p (1-p) \mathbf{x}_i$. We obtain $$\frac{\partial \ell}{\partial \boldsymbol{\beta}} = \sum_i^n \mathbf{x}_i^T (y_i - p_i)$$ (it takes a couple of lines to work through). We further get the second derivative as $$\frac{\partial \ell}{\partial \boldsymbol{\beta}\boldsymbol{\beta}^T} = \sum_i^n  p_i(1-p_i)\mathbf{x}_i\mathbf{x}_i^T$$. Now we can use Newton-Raphson to iteratively update the parameters: $$\boldsymbol{\beta}^{t+1} = \boldsymbol{\beta}^{t} - \left( \frac{\partial \ell}{\partial \boldsymbol{\beta}\boldsymbol{\beta}^T} \right)^{-1} \frac{\partial \ell}{\partial \boldsymbol{\beta}}$$ where the derivatives are evaluated at $\boldsymbol{\beta}^{t}$.

We can further write this as follows: $$\frac{\partial \ell}{\partial \boldsymbol{\beta}} = \mathbf{X}^T(\mathbf{y}-\mathbf{p})$$,
$$\frac{\partial \ell}{\partial \boldsymbol{\beta}\boldsymbol{\beta}^T}= -\mathbf{X}^T \mathbf{W} \mathbf{X}$$ where $\mathbf{W} = \mathrm{diag}(\mathbf{p}(1-\mathbf{p}))$ that is a diagonal matrix with $p_i(1-p_i)$ along its diagonal. Then an update can be written as $$\boldsymbol{\beta}^{t+1} = \boldsymbol{\beta}^{t} + (\mathbf{X}^T \mathbf{W} \mathbf{X})^{-1}\mathbf{X}^T(\mathbf{y}-\mathbf{p})$$ which after rearranging can be written as $$\boldsymbol{\beta}^{t+1}=(\mathbf{X}^T \mathbf{W} \mathbf{X})^{-1}\mathbf{X}^T\mathbf{W}(\mathbf{X}\boldsymbol{\beta}^{t}+\mathbf{W}^{-1}(\mathbf{y}-\mathbf{p}))$$. This is also known as the iteratively reweighted least squares algorithm (IRLS) because it has the form of a least squares update on a transformed response. A good starting value for the $\beta_j$ values is zero.

In [6]:
X_ = np.concatenate([np.ones((N,1)), X], axis=1)
sigmoid = lambda z: 1/(1+np.exp(-z))

In [7]:
max_iter = 100
epsilon = 0.1
beta = np.zeros((p+1,))
for _ in range(max_iter):
    prob = sigmoid(np.dot(X_, beta))
    print(beta)
    W = np.diag(prob*(1-prob))
    old_beta = beta.copy()
    beta += np.dot(inv(np.dot(np.dot(X_.T, W), X_)), np.dot(X_.T, y-prob))
    loglik = np.sum(y*np.log(prob)+(1-y)*np.log(1-prob))
    print(loglik)
    if np.sum(np.abs(old_beta - beta)) < epsilon:
        break

[0. 0. 0. 0.]
-13.862943611198906
[-0.69712367  0.75797279 -1.18168976  0.73446252]
-3.085462595608325
[-1.09474536  1.20285868 -1.90649182  1.24847506]
-1.1478580182013316
[-1.46458468  1.63291359 -2.61258951  1.77757897]
-0.44281074096931144
[-1.84315806  2.08024807 -3.33447255  2.33079118]
-0.1716131772388498
[-2.2474597   2.55513689 -4.08152384  2.90718279]
-0.06627108741433989
[-2.68445507  3.06015924 -4.85587836  3.50374865]
-0.025434057020906203
[-3.15381521  3.59364408 -5.65655077  4.11698952]
-0.009696397576888463
[-3.65008844  4.15122336 -6.48106017  4.74354061]
-0.0036736819117452304
[-4.16496947  4.72713553 -7.32637813  5.38043293]
-0.0013843279479963423
[-4.68909123  5.31520827 -8.18951394  6.0250999 ]
-0.0005192735486494641
[-5.21307021  5.90941993 -9.06781254  6.67528401]
-0.00019404923480903117
[-5.72798643  6.50414714 -9.95904775  7.32895687]
-7.228925125446261e-05
[ -6.22564102   7.09428498 -10.86139888   7.98429023]
-2.6860872568431887e-05
[ -6.69888969   7.67540088 

/Users/larshertel/python3env/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log
  # Remove the CWD from sys.path while we load stuff.
/Users/larshertel/python3env/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in multiply
  # Remove the CWD from sys.path while we load stuff.


LinAlgError: Singular matrix

In [14]:
beta

array([nan, nan, nan, nan])

## Naive Bayes
To do: add discrete feature

In [20]:
# [Mean(X|Y=0), Mean(X|Y=1)], shape=(p,2)
means = np.vstack([X[y==0].mean(0), X[y==1].mean(0)]).T
# [Var(X|Y=0), Var(X|Y=1)], shape=(p,2)
variances = np.vstack([X[y==0].var(0), X[y==1].var(0)]).T

In [22]:
def gaussian(x,m,v):
    return np.exp(-(x-m)**2/(2*v))/np.sqrt(2*np.pi*v)

In [36]:
# [P(Y=0), P(Y=1)], shape=(2,)
prevalence = np.array([1-y.mean(), y.mean()])

In [51]:
# Note: X.shape = (20,3), means.shape = (3,2)
# Add axis 2 to X to broadcast the last axis of means (that is classes)
# Add axis 0 to means and variances to broadcast of the zeroeth axis of X (that is data points)
def naive_bayes(X_):
    return np.argmax(gaussian(X_[:,:,None],means[None,:,:],variances[None,:,:]).prod(1) * prevalence,
                     axis=1)